In [62]:
!git clone https://github.com/ntienhuy/MultiChannel

fatal: destination path 'MultiChannel' already exists and is not an empty directory.


In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
import torch 
import numpy as np 
import re 
MAX_SEQ_LEN = 100
BATCH_SIZE = 64
EMBEDDING_DIM = 400

In [65]:
# Ham xu ly chuỗi sai dấu
# ví dụ:  'Nhe ̣ nha ̀ ng sâu lă ́ ng nhưng cu ̃ ng triê ́ t ly ́ va ̀ tinh tê ́ .'
def chuan_hoa_dau(s):
    dau_nang_sai = ' ̣ '
    dau_nang_dung = dau_nang_sai[1]
    dau_huyen_sai = ' ̀ '
    dau_huyen_dung = dau_huyen_sai[1]
    dau_sac_sai = ' ́ '
    dau_sac_dung = dau_sac_sai[1]
    dau_nga_sai = ' ̃ '
    dau_nga_dung = dau_nga_sai[1]
    dau_hoi_sai = ' ̉ '
    dau_hoi_dung = dau_hoi_sai[1]
    dau_cau = {dau_nang_sai : dau_nang_dung,
           dau_huyen_sai : dau_huyen_dung,
           dau_nga_sai : dau_nga_dung,
           dau_sac_sai : dau_sac_dung,
           dau_hoi_sai : dau_hoi_dung}
    nguyen_am = 'aáàạảãăắằằẳẵâấầậẩẫeéèẻèẽêếềểễệiỉíìĩịoóòỏõọôốồộổỗơớờởỡợuúùụủũưứừựửữyýỳỷỹỵ'

    for k, v in dau_cau.items():
        while(s.find(k) >= 0):
            pos = s.find(k)
            s1 = s[pos+3:]
            # s2 là những ký tự phía sau dấu câu sai
            s2 = s1[:s1.find(' ')]
            if len(s2) == 2:
                #phía sau là môt từ
                if (nguyen_am.find(s2[0])>=0) | (nguyen_am.find(s2[1])>=0):
                    s=s.replace(k,v + ' ',1)
                #phia sau k phai 1 tu
                else: 
                    s=s.replace(k,v,1)
            #phia sau la mot tu
            elif len(s2) > 2:
                s=s.replace(k,v + ' ',1)
            #phia sau chi co 1 ky tu va no la 1 tu dac biet ngan
            elif 'áàạéèẻèêếềiíịoóòỏôốồơớờởợuúùụủũưứừựyýỳỵ'.find(s)>0:
                s=s.replace(k,v + ' ',1)
            else:
                s=s.replace(k,v,1)
    return s


In [66]:
!gdown --id 1TzXkIicD3MsI6ciKnsDfSX4vFmYqd3LV
#download teencode.txt

Downloading...
From: https://drive.google.com/uc?id=1TzXkIicD3MsI6ciKnsDfSX4vFmYqd3LV
To: /content/teencode.txt
100% 5.66k/5.66k [00:00<00:00, 5.12MB/s]


In [67]:
f = open('/content/teencode.txt','r', encoding='UTF-8')
teencode_dict = {}
for aline in f.readlines():
    aline = aline.split()
    teencode_dict[aline[0]] = ' '.join(aline[1:])
# teencode_dict

In [68]:
def load_data(folder_id):
    train_data = []
    train_labels = []    
    test_data = []
    test_labels = []
    for label in range(3):

        # read train data
        path = '/content/MultiChannel/data/VS/Data_Token/Fold_{}/train_nhan_{}.txt'.format(str(folder_id),str(label))
        f = open(path, "r",encoding='utf-8-sig')
        regex = re.compile('[-:()\",.!?=`~/+&$%^*#]')
        text = []
        for aline in f.readlines():
            aline = chuan_hoa_dau(aline)
            aline = aline.lower()
            aline = regex.sub('', aline).strip()
            aline = re.sub(' +',' ',aline)
            for k,v in teencode_dict.items():
                aline.replace(k,v)
            if len(aline.split())==0:
                continue
            text.append(aline)
        train_labels += [label]*len(text)
        print('label: {}, len: {}'.format(label, len(text)))
        train_data +=text
        f.close()

        # read test data
        path = '/content/MultiChannel/data/VS/Data_Token/Fold_{}/test_nhan_{}.txt'.format(str(folder_id),str(label))
        f = open(path, "r",encoding='utf-8-sig')
        regex = re.compile('[-:()\",.!?=`~/+&$%^*#]')
        text = []
        for aline in f.readlines():
            aline = chuan_hoa_dau(aline)
            aline = aline.lower()
            aline = regex.sub('', aline).strip()
            aline = re.sub(' +',' ',aline)
            for k,v in teencode_dict.items():
                aline.replace(k,v)
            if len(aline.split())==0:
                continue
            text.append(aline)
        test_labels += [label]*len(text)
        test_data += text
        f.close()



    return train_data, train_labels, test_data, test_labels

In [2]:
folder_id = 1
train_data, train_labels, test_data, test_labels = load_data(folder_id=folder_id)

In [70]:
print(len(train_data))
print(len(train_labels))
print(len(test_data))
print(len(test_labels))

29076
29076
5883
5883


In [71]:
!pip install pytorch-nlp

In [72]:
from torchnlp.encoders.text import StaticTokenizerEncoder,pad_tensor,stack_and_pad_tensors

encoder = StaticTokenizerEncoder(train_data, tokenize=lambda s: s.split())

train_encoded = [encoder.encode(aline) for aline in train_data]
train_input_len = [len(x) if len(x) < MAX_SEQ_LEN else MAX_SEQ_LEN for x in train_encoded]
train_padded = [pad_tensor(x, length=MAX_SEQ_LEN) if len(x)<MAX_SEQ_LEN else x[-MAX_SEQ_LEN:] for x in train_encoded]
train_padded = stack_and_pad_tensors(train_padded)

test_encoded = [encoder.encode(aline) for aline in test_data]
test_input_len = [len(x) if len(x) < MAX_SEQ_LEN else MAX_SEQ_LEN for x in test_encoded]
test_padded = [pad_tensor(x, length=MAX_SEQ_LEN) if len(x)<MAX_SEQ_LEN else x[-MAX_SEQ_LEN:] for x in test_encoded]
test_padded = stack_and_pad_tensors(test_padded)

In [73]:
print(max(train_input_len))
print(max(test_input_len))

100
100


In [74]:
train_labels = torch.Tensor(train_labels)
train_input_len = torch.Tensor(train_input_len)
dataset = torch.utils.data.TensorDataset(train_padded[0], train_input_len , train_labels)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,
                                           shuffle=True)

test_labels = torch.Tensor(test_labels)
test_input_len = torch.Tensor(test_input_len)
dataset = torch.utils.data.TensorDataset(test_padded[0], test_input_len , test_labels)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,
                                          shuffle=True)

In [76]:
#download word2vec
!gdown --id 19wlnUlDGtQc0p3XCBKudlGf4pW4C53_d

Downloading...
From: https://drive.google.com/uc?id=19wlnUlDGtQc0p3XCBKudlGf4pW4C53_d
To: /content/baomoi.model.bin
708MB [00:05, 127MB/s]


In [77]:
from gensim.models import KeyedVectors
import random
def read_embedding(word2vec_path):
    weights = []
    word_to_index = {}
    word2vec = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

    word_to_index['<unk>'] = len(weights)
    weights.append([random.uniform(0, 1) for _ in range(400)])
    word_to_index['<pad>'] = len(weights)
    weights.append([0 for _ in range(400)])

    for word in word2vec.wv.vocab:
        word_to_index[word] = len(weights)
        weights.append(word2vec.get_vector(word))

    return word_to_index, torch.tensor(weights, dtype=torch.float32)
    
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.from_pretrained(torch.FloatTensor(weights_matrix))
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [78]:
word_to_index, weights = read_embedding('/content/baomoi.model.bin')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  del sys.path[0]


In [79]:
weights.size()

torch.Size([439058, 400])

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable 
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

torch.manual_seed(1)

In [83]:
class MyLSTM(nn.Module):
    def __init__(self, num_classes,  hidden_size, num_layers,weights_matrix):
        super(MyLSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.hidden_size = 128
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix,True)


        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden_size,
                            num_layers = num_layers, bidirectional = False, batch_first = True)
        self.fc_1 = nn.Linear(self.hidden_size, 64)
        self.fc_2 = nn.Linear(64,num_classes)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self,x,x_length):
        batch_size = x.shape[0]
        # len = torch.sum(x != 0, axis=1) -1
        x = self.embedding(x)
        x =pack_padded_sequence(x, x_length.tolist(), batch_first=True,enforce_sorted=False)
        x, (h_n, c_n)  = self.lstm(x)
        # x,_  = pad_packed_sequence(x, batch_first=True)
        # x = x[torch.arange(x.shape[0]),len,:]
        x = h_n[-1].view(batch_size,-1)
        x = self.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x

In [84]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, test_loader):
    # model.load_state_dict(torch.load('/content/drive/MyDrive/sentiment_analysis/folder{}_baomoiEmb.pth'.format(folder_id)))
    print('hi!!!')
    highest_acc = 0.7
    for epoch in range(1,n_epochs+1):
        model.train()
        loss_train = 0.0
        for inputs, inputs_len , labels in train_loader:
            inputs, inputs_len = inputs.type(torch.LongTensor), inputs_len.type(torch.LongTensor)
            labels = labels.type(torch.LongTensor)
            output = model(inputs.to(device),inputs_len.to(device))
            optimizer.zero_grad()
            loss = F.cross_entropy(output, labels.to(device))
            loss.backward()
            optimizer.step()
            loss_train +=loss.item()
            
        correct = 0
        # break
        model.eval()
        with torch.no_grad():
            for data in test_loader:
                input, input_len, label = data
                input, input_len = input.type(torch.LongTensor), input_len.type(torch.LongTensor)
                label = label.type(torch.LongTensor)
                output = model(input.to(device),input_len.to(device))
                predicted = torch.argmax(output,1)
                c = (predicted == label.to(device))#.squeeze()
                correct += c.sum()

        if epoch == 1 or epoch % 1 == 0:
            print('Epoch {}, Training loss {}, Val accuracy {}'.format(
                epoch,
                loss_train / len(train_loader),
                correct / len(test_labels)))
            
        if correct/len(test_labels) > highest_acc:
            highest_acc = correct/len(test_labels)
            print('new highest accuracy: ', highest_acc)
            torch.save(model.state_dict(), "/content/drive/MyDrive/sentiment_analysis/folder{}_baomoiEmb.pth".format(folder_id))
    print(highest_acc)



In [85]:
def get_default_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device = get_default_device()

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=False)
class DevicedataLoader():
    def __init__(self,dl,device):
        self.dl = dl
        self.device = device
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    def __len__(self):
        return len(self.dl)
print(device)

cuda


In [86]:
learning_rate = 1e-4

input_size = EMBEDDING_DIM
hidden_size = EMBEDDING_DIM

num_layers = 5 #number of stacked lstm layers

num_classes = 3 #number of output classes

In [87]:
lstm_model = MyLSTM(num_classes, hidden_size, num_layers,weights)
lstm_model.to(device)
optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [88]:
train_loader = DevicedataLoader(train_loader,device)
test_loader = DevicedataLoader(test_loader,device)

In [1]:

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = lstm_model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    test_loader = test_loader
)

# baomoiEmb
- folder 1: 0.7319
- folder 2: 0.7621
- folder 3: 0.7891
- folder 4: 0.8024
- folder 5: 0.7894